In [16]:
import torch, torchvision, PIL, numpy as np
import pathlib
import PIL
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
from tqdm.auto import tqdm
import math
from torchvision import models, transforms

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
dataset_dir = "/content/drive/My Drive/CPEN291 Project/data"

In [19]:
# Copied from SimpleModel.ipynb
class PricePredictionDataset:
    def __init__(self, root_dir, transform=None, transform_label=None):
        root_dir = pathlib.Path(root_dir).resolve()
        self.root_dir = root_dir
        self.transform = transform
        self.transform_label = transform_label

        self.labels_f = pd.read_csv(root_dir / 'all_labels.csv')

        def get_price(fn):
            fn = str(fn).rsplit('/', 1)[-1]
            row = self.labels_f.loc[self.labels_f['name'] == fn]
            index = row.index[0]
            return math.log(row['price'][index]) # log because we are scientists

        self.fns_labels = [(imgfn, get_price(imgfn))
                           for imgfn in sorted(root_dir.glob('*.jpg'))]

    def __len__(self):
        return len(self.fns_labels)

    def __getitem__(self, i):
        if torch.is_tensor(i):
            i = i.item()
        imgfn, label = self.fns_labels[i]
        img = PIL.Image.open(imgfn)
        if (len(img.split()) > 3): # some images have one more channel
          r,g,b, _ = img.split()
          img = PIL.Image.merge('RGB', (r,g,b))
        if (len(img.split()) == 1): # some images only have one channel
          img = img.convert('RGB')
        if self.transform:
            img = self.transform(img)
        if self.transform_label:
            label = self.transform_label(label)
        return (img, label)

In [20]:
# Copied from SimpleModel.ipynb
def show_img_price(sample):
    img, price = sample
    if torch.is_tensor(img):
        img = img.cpu().permute(1,2,0)
    plt.imshow(img)
    plt.title(math.e**price, color='black') # add color='w' arg if using a dark background
    plt.axis('off')

In [21]:
xform = torchvision.transforms.Compose([torchvision.transforms.Resize((224,224)), transforms.ToTensor()])
dataset_full= PricePredictionDataset(dataset_dir, transform=xform)

In [22]:
n_all = len(dataset_full)
n_train = int(0.8*n_all)
n_test = n_all - n_train
rng = torch.Generator().manual_seed(291)
dataset_train, dataset_test = torch.utils.data.random_split(dataset_full, [n_train, n_test], rng)
len(dataset_train), len(dataset_test)

(3598, 900)

In [124]:
loader_train = torch.utils.data.DataLoader(dataset_train, batch_size = 32, shuffle=True)
loader_test = torch.utils.data.DataLoader(dataset_test, batch_size = 32, shuffle=True)

In [115]:
model = torchvision.models.resnet50(pretrained=True)
model.fc.in_features

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


2048

In [229]:
torch.manual_seed(291)
model = torchvision.models.resnet34(pretrained=True)
model.fc = torch.nn.Sequential(
    torch.nn.Dropout(),
    torch.nn.Linear(model.fc.in_features, 1),
    # torch.nn.ReLU(),
    # torch.nn.Dropout(),
    # torch.nn.Linear(300,200),
    # torch.nn.ReLU(),
    # torch.nn.Dropout(),
    # torch.nn.Linear(200,100),
    # torch.nn.ReLU(),
    # torch.nn.Dropout(),
    # torch.nn.Linear(100,1),
)
torch.nn.init.xavier_uniform_(model.fc[1].weight)
# torch.nn.init.xavier_uniform_(model.fc[4].weight)
# torch.nn.init.xavier_uniform_(model.fc[7].weight)
# torch.nn.init.xavier_uniform_(model.fc[10].weight)



device = torch.device('cuda:0')
model.to(device);

In [231]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=0.8)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
#scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01,steps_per_epoch=len(loader_train), epochs = 10)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=0.2,momentum=0.9)

In [232]:
def run_test(model, criterion, no_preds=False):
    nsamples_test = len(dataset_test)
    loss = 0
    preds = []
    model.eval()
    with torch.no_grad():
        for samples, labels in loader_test:
            samples = samples.to(device)
            labels = labels.to(device).float().unsqueeze(1)
            outs = model(samples)
            if not no_preds:
                preds += outs.cpu().unbind()
            loss += torch.sqrt(criterion(outs, labels)) * samples.size(0)
    return loss / nsamples_test, preds

def run_train(model, criterion, opt, sched):
    nsamples_train = len(dataset_train)
    loss_sofar = 0
    model.train()
    with torch.enable_grad():
        for samples, labels in tqdm(loader_train, desc='iters', leave=False):
            samples = samples.to(device)
            labels = labels.to(device).float().unsqueeze(1)
            opt.zero_grad()
            outs = model(samples)
            loss = torch.sqrt(criterion(outs, labels))
            loss.backward()
            opt.step()
            loss_sofar += loss.item() * samples.size(0)
            if (math.isnan(loss.item())):
              print("loss nan")
              print(loss)
    sched.step()
    return loss_sofar / nsamples_train

def run_all(model, criterion, optimizer, scheduler, n_epochs):
    for epoch in tqdm(range(n_epochs), desc='epochs'):
        loss_train = run_train(model, criterion, optimizer, scheduler)
        loss_test, _ = run_test(model, criterion, no_preds=True)
        tqdm.write(f"epoch {epoch+1}: train loss {loss_train:.4f}, test loss {loss_test:.4f}")
    return loss_test

In [233]:
run_all(model,criterion,optimizer,scheduler,10)

epoch 1: train loss 1.8627, test loss 1.1237


epoch 2: train loss 1.2497, test loss 1.1659


epoch 3: train loss 1.2371, test loss 1.2966


epoch 4: train loss 1.2414, test loss 1.1251


epoch 5: train loss 1.2622, test loss 1.1444


...

epoch 7: train loss 1.1869, test loss 1.1306


epoch 8: train loss 1.1691, test loss 1.1256


epoch 9: train loss 1.1691, test loss 1.1322


epoch 10: train loss 1.1736, test loss 1.1511



tensor(1.1511, device='cuda:0')

| model | fc | crit |batch size| optimizer | lr | momentum | wd | scheduler | step size | gamma| b train loss | b test loss |epochs| b epoch| notes|
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |--- |--- |--- |--- |
|resnet18|3 linear (in->300->100->1) w/ Dropout and ReLU|mse|32|AdamW|0.01|n/a|0.1|StepLR|3|0.1|3.1278|1.6563|10|9|Seems to have some generalization error maybe simplify model|
|resnet18|2 linear (in->256->1) w/Dropout and ReLU|mse|32|AdamW|0.01|n/a|0.1|StepLR|3|0.1|12.3806|4.2316|10|6| Still has generalization error but also, high loss, maybe need more complex model, but change other params|
|resnet18|3 linear (in->250->120->1) w/Dropout and ReLU| mse|32|AdamW| 0.01|n/a|0.1|StepLR|5|0.1|3.6726|8.3442|10|4|Test loss seemed to bounce around at really large values, maybe make model more complex
|resnet18| 4 linear(in->300->200->100->1) w/Dropout and ReLU|mse|32|AdamW|0.01|n/a|0.1|StepLR|5|0.1|2.9023|1.3666|10|7|Improvement over first trial, still not great test or train loss maybe try different scheduler|
|resnet18| 4 linear(in->300->200->100->1) w/Dropout and ReLU|mse|32|AdamW|0.001|n/a|0.1|OneCycleLR|n/a (steps_per_epoch=len(loader_train)) epochs = 10|n/a (max lr =0.01)|n/a|n/a|5|n/a|Started overfitting after 5 epochs, maybe try different optimizer with old scheduler.
|resnet18|4 linear(in->300->200->100->1) w/Dropout and ReLU|mse|32|SGD|0.001|0.9|0.4|StepLR|5|0.1|n/a|n/a|5|n/a|After 5 epochs, not noticable imrpovement in either test or train loss need to adjust params
|resnet18|4 linear(in->300->200->100->1) w/Dropout and ReLU|mse|16|SGD|0.01|0.9|0.5|StepLR|3|0.1|n/a|n/a|5|n/a|For some reason, using SGD seems to keep test and train loss around 60+, maybe try AdamW again with diff parameters
|resnet18|4 linear(in->300->200->100->1) w/Dropout and ReLU|mse|16|AdamW|0.001|n/a|0.2|StepLR|3|0.1|3.9388|11.0161|10|3|Model overfitting, need to increase wd, also 32 batchsize seems favourable. Also train loss does still high
|resnet18|4 linear(in->300->200->100->1) w/Dropout and ReLU|mse|32|AdamW|0.005|n/a|0.7|StepLR|3|0.1|3.6564|1.3145|10|4|Train loss, improving slowly, but test loss not, appears to be overfitting still.
|resnet18|4 linear(in->300->200->100->1) w/Dropout and ReLU|mse|32|AdamW|0.0075|n/a|0.9|StepLR|3|0.1|3.9773|1.3475|10|4|Same as previous attempt, overfitting maybe try completely new idea|
|resnet18|1 linear(in->1) w/Dropout|mse|32|AdamW|0.0075|n/a|0.9|StepLR|3|0.1|1.3806|1.2703|10|10|Seems better than 4 linear, but still not great loss, could be better with more epochs|
|resnet18|1 linear(in->1) w/Dropout|mse|32|AdamW|0.01|n/a|0.9|StepLR|2|0.1|1.4018|1.2943|10|6|Worse loss than previous model|
|resnet18|1 linear(in->1) w/Dropout|mse|32|AdamW|0.01|n/a|0.95|StepLR|3|0.1|1.3682|1.2848|10|10|Compareable loss to previous models, maybe try different base model|
|resnet34|1 linear(in->1) w/Dropout|mse|32|AdamW|0.0075|n/a|0.9|StepLR|3|0.1|1.3625|1.2618|10|10|Better performance than resnet18 with same parameters|
|resnet34|1 linear(in->1) w/Dropout|mse|32|AdamW|0.0075|n/a|0.3|StepLR|3|0.1|1.4775|1.2880|10|2|Model overfitting|
|resnet34|1 linear(in->1) w/Dropout|mse|32|AdamW|0.0075|n/a|0.5|StepLR|3|0.1|1.4408|1.2830|10|10|Worse loss than previous models|
|resnet34|1 linear(in->1) w/Dropout|mse|32|AdamW|0.0075|n/a|0.8|StepLR|3|0.1|1.3435|1.2577|10|7|Improvement over previous models|
|resnet34|1 linear(in->1) w/Dropout|mse|32|AdamW|0.009|n/a|0.8|StepLR|3|0.1|1.3446|1.2205|10|10|Best loss so far|
|resnet34|1 linear(in->1) w/Dropout|mse|32|AdamW|0.01|n/a|0.8|StepLR|3|0.1|1.4030|1.2840|10|10|Worse loss, on increase learning rate maybe lower is better|
|resnet34|1 linear(in->1) w/Dropout|mse|32|AdamW|0.0095|n/a|0.8|StepLR|5|0.1|1.3744|1.2739|10|8|Increased stepsize doesnt seem to be better|
|resnet34|1 linear(in->1) w/Dropout|mse|32|AdamW|0.009|n/a|0.8|StepLR|3|0.05|1.3858|1.2597|10|10|Decreasing gamma did not appear to help|
|resnet34|1 linear(in->1) w/Dropout|mse|32|AdamW|0.009|n/a|0.8|StepLR|3|0.2|1.3554|1.2336|10|10|Loss comparable to best loss so far from increasing gamma|
|resnet34|1 linear(in->1) w/Dropout|rmse|32|AdamW|0.009|n/a|0.8|StepLR|3|0.1|1.1546|1.1193|10|10|First instance of using root-mean-square for criterion resulted in best loss|
|resnet34|1 linear(in->1) w/Dropout|rmse|32|AdamW|0.01|n/a|0.8|StepLR|5|0.1|1.1691|1.1256|10|8|Not terrible compared to models thats use rmse but not an improvement|


